In [9]:
import numpy as np
import scipy.linalg as linalg
from dtuimldmtools import similarity
import matplotlib.pyplot as plt
from scipy.linalg import svd
from pandas.plotting import scatter_matrix
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pyplot import figure, legend, plot, show, xlabel, ylabel
# exercise 8.1.1
from dtuimldmtools import dbplotf, train_neural_net, visualize_decision_boundary
import importlib_resources
import numpy as np
import torch
from matplotlib.pyplot import figure, show, title
from scipy.io import loadmat
import importlib_resources
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pylab import (
    figure,
    grid,
    legend,
    loglog,
    semilogx,
    show,
    subplot,
    title,
    xlabel,
    ylabel,
)
from scipy.io import loadmat
from sklearn import model_selection

from dtuimldmtools import rlr_validate

# fetch dataset 
# wine = fetch_ucirepo(id=109) 
  
# # data (as pandas dataframes) 
# X = wine.data.features 
# y = wine.data.targets 

# totaldata= (wine.data)
  
# metadata 
# print(wine.metadata) 
  
# variable information 
# print(wine.variables) 

# OFFLINE LOADING OF DATA
X = np.loadtxt('../wine/wine.data', delimiter=',')

#Extract classes from X
y = X[:,0]
X = np.delete(X,0,axis=1)
# y = np.loadtxt('../wine/wine.names', delimiter=',')

Xorig = X
# Standardizing the data

X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)



attributeNames = [
    "Alcohol",
    "Malic acid",
    "Ash",
    "Alcalinity of ash",
    "Magnesium",
    "Total phenols",
    "Flavanoids",
    "Nonflavanoid phenols",
    "Proanthocyanins",
    "Color intensity",
    "Hue",
    "OD280/OD315 of diluted wines",
    "Proline"
]

N, M = X.shape

# Add offset attribute
attributeNames = ["Offset"] + attributeNames
# M = M + 1


classNames = ["1", "2", "3"]


In [20]:
# %% ANN classification


K = 10
CV = model_selection.KFold(K, shuffle=True)
# Initialize variables
# T = len(lambdas)
Error_train = np.empty((K, 1))
Error_test = np.empty((K, 1))
Error_train_rlr = np.empty((K, 1))
Error_test_rlr = np.empty((K, 1))
Error_train_nofeatures = np.empty((K, 1))
Error_test_nofeatures = np.empty((K, 1))
w_rlr = np.empty((M, K))
mu = np.empty((K, M - 1))
sigma = np.empty((K, M - 1))
w_noreg = np.empty((M, K))

# Convert class labels to 0, 1, 2
y = y - 1

for train_index, test_index in CV.split(X, y):
    Error_train_avg = []
    Error_test_avg = []
    # extract training and test set for current CV fold
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    internal_cross_validation = 10


    # Define the model structure
    n_hidden_units = 5  # number of hidden units in the signle hidden layer
    C = 3
    model = lambda: torch.nn.Sequential(
        torch.nn.Linear(M, n_hidden_units),  # M features to H hiden units
        torch.nn.ReLU(),  # 1st transfer function
        # Output layer:
        # H hidden units to C classes
        # the nodes and their activation before the transfer
        # function is often referred to as logits/logit output
        torch.nn.Linear(n_hidden_units, C),  # C logits
        # To obtain normalised "probabilities" of each class
        # we use the softmax-funtion along the "class" dimension
        # (i.e. not the dimension describing observations)
        torch.nn.Softmax(dim=1),  # final tranfer function, normalisation of logit output
    )
    # Since we're training a multiclass problem, we cannot use binary cross entropy,
    # but instead use the general cross entropy loss:
    loss_fn = torch.nn.CrossEntropyLoss()
    # Train the network:
    # C = 3
    net, _, _ = train_neural_net(
        model,
        loss_fn,
        X=torch.tensor(X_train, dtype=torch.float),
        y=torch.tensor(y_train, dtype=torch.long),
        n_replicates=3,
        max_iter=10000
    )
    # Determine probability of each class using trained network
    softmax_logits = net(torch.tensor(X_test, dtype=torch.float))
    # Get the estimated class as the class with highest probability (argmax on softmax_logits)
    y_test_est = (torch.max(softmax_logits, dim=1)[1]).data.numpy()
    # Determine errors
    e = y_test_est != y_test
    print(
        "Number of miss-classifications for ANN:\n\t {0} out of {1}".format(sum(e), len(e))
    )

    predict = lambda x: (
        torch.max(net(torch.tensor(x, dtype=torch.float)), dim=1)[1]
    ).data.numpy()
    # figure(1, figsize=(9, 9))
    # visualize_decision_boundary(
    #     predict, [X_train, X_test], [y_train, y_test], attributeNames, classNames
    # )
    # title("ANN decision boundaries")

# show()

print("Ran Exercise 8.3.1")


	Replicate: 1/3
		Iter	Loss			Rel. loss


IndexError: Target -1 is out of bounds.